# Skyview masking

Create skyview mask from fisheye photo. The mask should be simplified, can be fuzzy. Workflow:

1. Open images
2. Find tresholds for sky, use blue, possible also red and green band.
3. Create fuzzy mask
4. Create hard mask
5. Fill smaller areas

In [ ]:
# Libraries
from skimage import io
from skimage import filters
from skimage import morphology
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
plt.rcParams['figure.figsize'] = (15, 10)

## Read image

In [ ]:
# Photo filename
photo = "./SkyView/2019_03_31.jpg"
photo = "./SkyView/2019_04_10.jpg"
# photo = "./SkyView/2019_04_12.jpg"
# photo = "./SkyView/2019_04_15.jpg"
image = io.imread(photo)

In [ ]:
image.shape

In [ ]:
plt.imshow(image)

In [ ]:
# Plot image, histogram, treshold, mask
def treshold_plot_hist (band, tresh, mask, title = ""):
    fig, axes = plt.subplots(ncols=3, figsize=(15, 5))
    fig.suptitle(title)
    
    ax = axes.ravel()
    ax[0] = plt.subplot(1, 3, 1)
    ax[1] = plt.subplot(1, 3, 2)
    ax[2] = plt.subplot(1, 3, 3, sharex=ax[0], sharey=ax[0])

    ax[0].imshow(band, cmap=plt.cm.gray)
    ax[0].set_title('Original')
    ax[0].axis('off')

    ax[1].hist(band.ravel(), bins=256)
    ax[1].set_title('Histogram')
    ax[1].axvline(tresh, color='r')
    ax[1].text(0.8, 0.95, tresh, transform=ax[1].transAxes, fontsize=14,
        verticalalignment='top', color='r')
    

    ax[2].imshow(mask, cmap=plt.cm.gray)
    ax[2].set_title('Thresholded')
    ax[2].axis('off')

    plt.show()

In [ ]:
# Plot image, mask
def treshold_plot (band, mask, title = ""):
    fig, axes = plt.subplots(ncols=2, figsize=(15, 7))
    fig.suptitle(title)
    
    ax = axes.ravel()
    ax[0] = plt.subplot(1, 2, 1)
    ax[1] = plt.subplot(1, 2, 2, sharex=ax[0], sharey=ax[0])

    ax[0].imshow(band, cmap=plt.cm.gray)
    ax[0].set_title('Original')
    ax[0].axis('off')

    ax[1].imshow(mask, cmap=plt.cm.gray)
    ax[1].set_title('Thresholded')
    ax[1].axis('off')

    plt.show()

## Aggregation

All bands are tresholded and the mask is combined. We are using Otsu filter tresholds.

In [ ]:
# Find tresholds
tresh = np.empty(3, dtype=int)
for img_b in range(3):
    img = image[:,:,img_b]
    tresh[img_b] = filters.threshold_otsu(img)
tresh

### Create masks

Creeate mask for all bands and add the masks.

In [ ]:
# Dermined treshold
mask = np.copy(image[:,:,0]) * 0
for i in range(3):
    mask_add = image[:,:,i] >= tresh[i]
    mask[:,:] = mask + mask_add

### Fuzzy mask

In [ ]:
plt.imshow(mask)

### Hard mask

If any of the treasholds has been detected, the mask is true.

In [ ]:
mask_h = mask > 1
plt.imshow(mask_h)

In [ ]:
mask_h_im = Image.fromarray(mask_h * 255)
mask_h_im = mask_h_im.convert("L")
# mask_h_im.show()
photo_out = photo.rsplit('.', 1)[0] + "_mask"+ ".png"
mask_h_im.save(photo_out)